In [ ]:
import pandas as pd

In [ ]:
path = '/content/drive/MyDrive/Projet_Bootcamp/'

In [ ]:
df = pd.read_csv(path+'Fake_Kaggle/test.csv')

In [ ]:
df_fake_kaggle = pd.read_csv(path+'Fake_Kaggle/train.csv')
df_fake_kaggle = df_fake_kaggle[(df_fake_kaggle['class'] == 'Fake') | (df_fake_kaggle['class'] == 'Real')]
df_fake_kaggle['class'] = df_fake_kaggle['class'].apply(lambda x : 0 if x == 'Real' else 1)
df_val_fake_kaggle = pd.DataFrame(pd.read_csv(path+'Fake_Kaggle/test.csv').text,columns=['text'])

In [ ]:
def GetFileTest(file_name):
    df = pd.read_csv(file_name)
    df = df.loc[:,['text','label']]
    df.dropna(inplace=True)
    df['label'] = df['label'].apply(lambda x : 0 if x == 1 else 1)
    return df

df_train_fake_news_kaggle = GetFileTest(path+'Fake_News_Kaggle/train.csv')
df_val_fake_news_kaggle = pd.read_csv(path+'Fake_News_Kaggle/test.csv').loc[:,['text']]
df_val_fake_news_kaggle.dropna(inplace=True)

In [ ]:
def GetMappingLiarDataset(files_name):
    dfs = []
    for file_name in files_name:
        df_liar = pd.read_csv(path+file_name,delimiter='\t')
        df_liar_mapping = df_liar.iloc[:,[1,2]]
        df_liar_mapping.columns=['IsFakeNews','text']
        df_liar_mapping['label'] = df_liar_mapping.IsFakeNews.apply(lambda x: 0 if x=='false' else 1)
        df_liar_mapping.drop('IsFakeNews',axis=1,inplace=True)
        dfs.append(df_liar_mapping)
    df_test = pd.concat([dfs[1],dfs[2]],axis=0)
    return (dfs[0],df_test)
df_train_liar_dataset, df_test_liar_datasset = GetMappingLiarDataset(['Liar_Dataset/train.tsv','Liar_Dataset/test.tsv','Liar_Dataset/valid.tsv'])

<ipython-input-9-8f60d20eded3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liar_mapping['label'] = df_liar_mapping.IsFakeNews.apply(lambda x: 0 if x=='false' else 1)
<ipython-input-9-8f60d20eded3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liar_mapping.drop('IsFakeNews',axis=1,inplace=True)
<ipython-input-9-8f60d20eded3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
df_fake_misinfo = pd.read_csv(path+'Misinfo/DataSet_Misinfo_FAKE.csv')
df_fake_misinfo['label'] = df_fake_misinfo['text'].apply(lambda x:1)
df_fake_misinfo = df_fake_misinfo.loc[:,['text','label']]
df_true_misinfo = pd.read_csv(path+'Misinfo/DataSet_Misinfo_TRUE.csv')
df_true_misinfo['label'] = df_true_misinfo['text'].apply(lambda x:0)
df_true_misinfo = df_true_misinfo.loc[:,['text','label']]
df_true_misinfo.dropna(inplace=True)
df_misinfo = pd.concat([df_fake_misinfo,df_true_misinfo])

In [ ]:
df_fake = pd.read_csv(path+'Fake_Real_News_Kaggle/Fake.csv')
df_fake = pd.DataFrame(df_fake.text,columns=['text'])
df_fake['label'] = df_fake.text.apply(lambda x : 1)
df_real = pd.read_csv(path+'Fake_Real_News_Kaggle/True.csv')
df_real = pd.DataFrame(df_real.text)
df_real['label'] = df_real.text.apply(lambda x : 0)
df_real_news_kaggle = pd.concat([df_fake,df_real],axis=0)

In [ ]:
df_welfake = pd.read_csv(path +'Wel_Fake_Kaggle/WELFake_Dataset.csv')
df_welfake = df_welfake.loc[:,['text','label']]
df_welfake.dropna(inplace=True)

In [ ]:
df_train_input = pd.read_csv(path+'Input_Fake_News/train.csv',delimiter=';')
df_train_input = df_train_input.loc[:,['text','label']]
df_test_input = pd.read_csv(path+'Input_Fake_News/test.csv',delimiter=';')
df_test_input = df_test_input.loc[:,['text','label']]
df_evaluation_input = pd.read_csv(path+'Input_Fake_News/evaluation.csv',delimiter=';')
df_evaluation_input = df_evaluation_input.loc[:,['text','label']]

In [ ]:
df_final_train = pd.concat([df_train_fake_news_kaggle, df_train_liar_dataset, df_test_liar_datasset, df_misinfo, df_real_news_kaggle, df_welfake, df_train_input, df_test_input])

In [ ]:
import numpy as np

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    output = np.asarray(pos_tag)
    for i in range(len(pos_tag)):
        if pos_tag[i][1].startswith('J'):
            output[i][1] = wordnet.ADJ
        elif pos_tag[i][1].startswith('V'):
            output[i][1] = wordnet.VERB
        elif pos_tag[i][1].startswith('R'):
            output[i][1] = wordnet.ADV
        else:
            output[i][1] = wordnet.NOUN
    return output

def Tokenize(text):
    text = text.lower()
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    token_liste = [token for token in word_tokenize(text) if token.isalpha() and token not in stop_words]
    test = False
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    for mot in token_liste:
        for char in mot:
            if char not in alphabet:
                test = True
    if test == False:
        final_liste = [token for token in token_liste]
        tags = nltk.pos_tag(final_liste)
        wordnet_input = get_wordnet_pos(tags)
        lem_tokens = [lemmatizer.lemmatize(t,tag) for t,tag in wordnet_input]
        return lem_tokens
    return [' ']

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df_final_train['tokenize'] = df_final_train['text'].apply(Tokenize)

In [ ]:
! nvidia-smi

Thu Dec 12 18:54:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
df_final_train

,text,label,tokenize
0,House Dem Aide: We Didn’t Even See Comey’s Let...,0,"[house, dem, aide, even, see, comey, letter, j..."
1,Ever get the feeling your life circles the rou...,1,"[ever, get, feeling, life, circle, roundabout,..."
2,"Why the Truth Might Get You Fired October 29, ...",0,"[truth, might, get, fired, october, tension, i..."
3,Videos 15 Civilians Killed In Single US Airstr...,0,"[video, civilian, kill, single, u, airstrike, ..."
4,Print \nAn Iranian woman has been sentenced to...,0,"[print, iranian, woman, sentence, six, year, p..."
...,...,...,...
8112,VATICAN CITY (Reuters) - U.S. presidential can...,1,"[vatican, city, reuters, presidential, candida..."
8113,force and fanaticism wahhabism in saudi arabi...,1,"[force, fanaticism, wahhabism, saudi, arabia, ..."
8114,"ISTANBUL (Reuters) - Turkey, whose relations w...",1,"[istanbul, reuters, turkey, whose, relation, g..."
8115,Here s why Clinton and Abedin should be flippi...,0,"[clinton, abedin, flip, multiple, state, emplo..."


In [ ]:
df_final_train['tokenized_text'] = df_final_train['tokenize'].apply(lambda tokens:' '.join(tokens).lower())

In [ ]:
import pickle
with open(path+'df_final_train.pickle','wb') as file:
    pickle.dump(df_final_train,file)

In [2]:
import pandas as pd
path = '/content/drive/MyDrive/Projet_Bootcamp/'

In [3]:
import pickle
with open(path+'df_final_train.pickle','rb') as file:
    df_final_train = pickle.load(file)

In [4]:
df_final_train = df_final_train[df_final_train['tokenized_text'] != ' ']

In [5]:
df_cleaned = df_final_train.loc[:,['tokenized_text','label']]

In [6]:
from sklearn.model_selection import train_test_split
X = df_cleaned.tokenized_text
y = df_cleaned.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_final = vectorizer.fit_transform(X_train)
X_test_final = vectorizer.transform(X_test)

In [15]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression()
lreg.fit(X_train_final, y_train)
y_pred = lreg.predict(X_test_final)

In [14]:
y_pred

array([1, 1, 0, ..., 0, 1, 1])